In [186]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### 1. 대상지 주소 입력 > 법정동코드 추출

대상지 : 강원 철원군 갈말읍 군탄리 473-1 ( DMZ일원 화살머리고지 등 유해발굴지 )

In [187]:
from urllib.parse import quote
with open('road_key.txt') as file:
    road_key = f'{file.read()}'

In [188]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote(input())}&resultType=json'
url = f"{road_url}?{params1}&{params2}"
result = requests.get(url).json()

BJD_code=result['results']['juso'][0]['admCd'] 
BJD_code

'4278025624'

### 2. 법정동코드 > 대상지 기후대, 토양성분 추출

In [189]:
#https://www.forest.go.kr/kfsweb/kfi/kfs/trail/soil.do?mn=NKFS_06_08_02&fileDownload=Y&dataType=/soil/FS_IJ100_50.zip&pblicDataId=PBD0000181&tabs=1&searchSrvc=&subTitle=&searchWrd=&searchCnd=
addrr = pd.read_csv('전국.csv', sep=',')  #전국.csv만들기
addrr = pd.DataFrame(addrr)
addrr = addrr[['EMNDN_CD','CLZN_CD']]
addrr.columns = ['읍면동코드','기후대코드']
addrr['읍면동코드'] = int(BJD_code[:-2])
climate_code = addrr['기후대코드'].mode()[0]
climate_code  

# 1) 온대북부, 2)온대중부, 3)온대남부 4) 난대

# if climate_code == 1.0:
#     print('온대북부')
# elif climate_code == 2.0:
#     print('온대중부')
# elif climate_code == 3.0:
#     print('온대남부')
# elif climate_code == 4.0:
#     print('난대')

C:\Users\ICT01_11\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2.0

In [190]:
with open('soil_key.txt') as file:
    soilkey = file.read()

s_url = f'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey={soilkey}&Page_Size=10&Page_No=1&BJD_Code={BJD_code}'
s_result = requests.get(s_url)
s_contents = s_result.text
s_soup = BeautifulSoup(s_contents, 'html.parser')

acid = []
vldpha = []
vldsia = []
om = []
posifert_mg = []
posifert_k = []
posifert_ca = []

lis = [acid, vldpha, vldsia, om, posifert_mg, posifert_k, posifert_ca]
cons = ['acid', 'vldpha', 'vldsia', 'om', 'posifert_mg', 'posifert_k', 'posifert_ca']

for li in lis:
    for con in cons:
        if lis.index(li) == cons.index(con):
            for a in s_soup.find_all(con):
                li.append(a.get_text())

ground_con = pd.DataFrame({'산도':acid, '유효인산':vldpha, '유기물':om, '마그네슘':posifert_mg, '칼륨':posifert_k , '칼슘':posifert_ca})
ground_con

,산도,유효인산,유기물,마그네슘,칼륨,칼슘
0,5.6,49,25.9,1.1,.43,5.7
1,4.9,25.5,16.9,1,.26,4.8
2,5.7,56.1,22.8,1.2,.37,3.5
3,5.6,53.8,26.2,4,.5,7.3
4,6.2,63.4,20.3,3.8,.75,8.2
5,6.1,84.2,26.9,1.8,.41,6.8
6,5.9,141.1,35.4,1.4,.69,5.8
7,6.1,138.3,30.3,2.1,.64,9
8,6,224.4,37.9,1.9,.96,8.1
9,5.3,54.1,31.4,1.6,.49,6.3


In [197]:
# 지번코드 -> 
# 자생식물 주소-> 지번코드 
# 